In [ ]:
import os
import json
import base64
import time
import requests
import numpy as np
import torch
import torch.nn as nn
from PIL import Image
from io import BytesIO
from urllib.parse import urlparse
from datetime import datetime
# --------------------------------------------------------------------------------
# (1) Mobius 플랫폼 설정
# --------------------------------------------------------------------------------

# 수업용 데이터 AE (모델 + 이미지 데이터가 있는 곳)
MOBIUS_BASE_URL = "https://onem2m.iotcoss.ac.kr/Mobius/"
MOBIUS_DATA_AE_NAME = "AIoT-HW"  # 수업용 AE (수정 X)

MOBIUS_IMAGE_DATA_CNT_NAME = "image_data"           # 이미지 데이터 컨테이너
MOBIUS_MODEL_REPOSITORY_CNT_NAME = "model_repository"  # 모델 저장 컨테이너

MOBIUS_DATA_AE_URL = os.path.join(MOBIUS_BASE_URL, MOBIUS_DATA_AE_NAME)

# 학생용 AE (추론 결과를 업로드할 곳) - 본인 정보로 수정!
MOBIUS_STUDENT_AE_NAME = "AIoT-HW-학번"  # ← 본인 학번 이어붙여 생성
MOBIUS_STUDENT_INFERENCING_RESULT_CNT_NAME = "inferencing_result"
MOBIUS_STUDENT_AE_URL = os.path.join(MOBIUS_BASE_URL, MOBIUS_STUDENT_AE_NAME)
#실습 실행 이전에 개인용 AE와 추론 결과 저장용 CNT가 반드시 만들어져 있어야 함!

# GET 요청 헤더 (수업용 데이터 조회)
HEADERS_GET = {
    'Accept': 'application/json',
    'X-M2M-RI': '12345',
    'X-M2M-Origin': 'SOrigin',
    'X-API-KEY': 'your-api-key',              # ← 본인 API 키
    'X-AUTH-CUSTOM-LECTURE': 'lecture-id',    # ← 수업 ID
    'X-AUTH-CUSTOM-CREATOR': 'your-creator-id' # ← 본인 ID
}

# POST 요청 헤더 (추론 결과 업로드)
HEADERS_CIN = {
    'Accept': 'application/json',
    'X-M2M-RI': '12345',
    'X-M2M-Origin': 'SOriginHW학번',          # ← 본인 Originator, 학번 입력
    'Content-Type': 'application/vnd.onem2m-res+json; ty=4',
    'X-API-KEY': 'your-api-key',              # ← 본인 API 키
    'X-AUTH-CUSTOM-LECTURE': 'lecture-id',    # ← 수업 ID
    'X-AUTH-CUSTOM-CREATOR': 'your-creator-id' # ← 본인 ID
}
# --------------------------------------------------------------------------------
# (2) Mobius 연동 함수
# --------------------------------------------------------------------------------

def http_get(url, params=None, headers=None, iotPlatform=None):
    """
    주어진 URL에 GET 요청을 보내고, 결과를 JSON(dict)로 반환한다.
    iotPlatform=True이면 OneM2M용 기본 헤더가 적용된다.
    """
    if iotPlatform:
        headers = {
            'Accept':       'application/json',
            'X-M2M-RI':     '12345',
            'X-M2M-Origin': 'SOrigin',
            'X-API-KEY': 'your-api-key',             
            'X-AUTH-CUSTOM-LECTURE': 'lecture-id',    
            'X-AUTH-CUSTOM-CREATOR': 'your-creator-id' 
        }

    try:
        response = requests.get(url, params=params, headers=headers, timeout=10)
        response.raise_for_status()
        return json.loads(response.text)
    
    except requests.ConnectTimeout:
        print(f"[Error] Connection timed out for URL: {url}")
        return None

    except requests.HTTPError as http_err:
        print(f"[Error] HTTP error occurred for URL {url}: {http_err}")
        return None

    except Exception as err:
        print(f"[Error] An error occurred for URL {url}: {err}")
        return None


def all_cin_get_uri(path, max_retries=10):
    """
    주어진 컨테이너 경로에서 모든 콘텐츠 인스턴스(CIN) URI를 얻어
    각각을 GET하여 'con' 필드를 모아 리스트로 반환한다.
    """
    path = path + '?fu=1&ty=4'  # 모든 CIN(ResourceType=4) 조회 쿼리
    parsed_path = urlparse(path)
    base_path   = f"{parsed_path.scheme}://{parsed_path.netloc}/"
    resource_path = path.split('?')[0]

    con_list = []
    print(f"  [DISCOVERY] 대상 경로: {resource_path}")
    all_uri  = http_get(path, iotPlatform=True)
    if not all_uri:
        print(f"  [DISCOVERY] URI 목록을 가져오지 못했습니다: {resource_path}")
        return con_list  # 에러 시 빈 리스트 반환

    uri_list = all_uri.get("m2m:uril", [])
    total_uris = len(uri_list)
    print(f"  [DISCOVERY] 발견된 CIN URI: {total_uris}건")
    if not uri_list:
        return con_list

    # 각 URI에 대해 실제 데이터 GET
    for idx, uri in enumerate(uri_list, start=1):
        print(f"    [FETCH {idx}/{total_uris}] {uri}")
        retries = 0
        while retries < max_retries:
            cin = http_get(base_path + uri, iotPlatform=True)
            if cin is not None:
                con_list.append(cin["m2m:cin"]["con"])
                break
            else:
                retries += 1
                print(f"[Retry {retries}] for URL: {base_path + uri}")

        if retries == max_retries:
            print(f"[FAIL] Data not fetched after {max_retries} attempts for URL: {base_path + uri}")

    return con_list


def upload_data(container_name, data, base_url):
    """
    컨테이너에 데이터(CIN) 업로드
    """
    url = f"{base_url}/{container_name}"
    body = {
        "m2m:cin": {
            "con": data
        }
    }
    response = requests.post(url, headers=HEADERS_CIN, json=body)
    if response.status_code in [200, 201]:
        print(f"[OK] 업로드 성공: {container_name}")
    else:
        print(f"[FAIL] 업로드 실패: {response.text}")
        

def decode_model(base64_str, save_path="./mnist_cnn.pth"):
    """
    - Base64 문자열을 받아 .pth 모델 파일로 저장
    - 저장 경로 반환
    """
    decoded_bytes = base64.b64decode(base64_str)
    with open(save_path, "wb") as f:
        f.write(decoded_bytes)
    return save_path
# --------------------------------------------------------------------------------
# (3) 모델 정의 (예: MNISTCNN)
# --------------------------------------------------------------------------------
class MNISTCNN(nn.Module):
    def __init__(self, output_size=10):
        super(MNISTCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1   = nn.Linear(64 * 7 * 7, 128)
        self.fc2   = nn.Linear(128, output_size)
        self.pool  = nn.MaxPool2d(2, 2)
        self.relu  = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# --------------------------------------------------------------------------------
# (4) 이미지 전처리 함수 (Base64 → 텐서)
# --------------------------------------------------------------------------------
def preprocess_image_bytes(b64_str):
    """
    - Base64 문자열(b64_str)을 받아,
      1) base64 디코딩 → 2) BytesIO → 3) PIL.Image
      4) 28×28 흑백으로 변환 → 5) NumPy → 6) PyTorch 텐서
    - 최종 shape: (1, 1, 28, 28)
    """
    try:
        # 1) base64 → bytes
        img_bytes = base64.b64decode(b64_str)
        # 2) BytesIO → PIL
        pil_image = Image.open(BytesIO(img_bytes)).convert("L")
        # 3) 28×28 리사이즈 (MNIST)
        pil_image = pil_image.resize((28, 28))
        # 4) [0~255] 범위 유지 (정규화 제거)
        arr = np.array(pil_image).astype(np.float32)
        # 5) (H, W) → (1, 1, H, W)
        tensor = torch.from_numpy(arr).unsqueeze(0).unsqueeze(0)  # [1, 1, 28, 28]
        return tensor
    except Exception as e:
        print(f"[Error] preprocess_image_bytes failed: {e}")
        return None
# --------------------------------------------------------------------------------
# (5) 메인: Mobius 모델 로드 + 추론 후 결과 업로드 
# --------------------------------------------------------------------------------       
if __name__ == "__main__":

    # 1) Mobius에서 최신 모델 가져오기
    model_repo_path = f"{MOBIUS_DATA_AE_URL}/{MOBIUS_MODEL_REPOSITORY_CNT_NAME}"
    con_list = all_cin_get_uri(model_repo_path)
    if not con_list:
        print("[Error] No model data in Mobius.")
        exit(1)

    # 1.1) 가장 최근에 올라온 모델(con_list[-1]) 사용
    latest_model_data = con_list[-1] 
    base64_model_str  = latest_model_data["model_file"]
    metadata          = latest_model_data["metadata"]

    # 1.2) 디코딩 → 로컬에 저장
    model_path = decode_model(base64_model_str, save_path="./mnist_cnn.pth")

    # 1.3) 모델 로드
    model = MNISTCNN(output_size=10)
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu'), weights_only=True))
    model.eval()

    print(f"[MODEL SERVER] Loaded model: {metadata['model_name']} (version {metadata['version']})")

    # 2) Mobius에서 이미지 데이터 가져오기
    image_data_url = f"{MOBIUS_DATA_AE_URL}/{MOBIUS_IMAGE_DATA_CNT_NAME}"
    image_con_list = all_cin_get_uri(image_data_url)

    if not image_con_list:
        print("[Error] No image data in Mobius.")
        exit(1)

    print(f"[OK] 이미지 {len(image_con_list)}개 로드 완료")

    # 3) 각 이미지에 대해 추론 수행 및 결과 업로드
    for idx, con in enumerate(image_con_list, start=1):
        # con 구조: {"timestamp": "...", "raw_base64": "..."}
        b64_str = con.get("raw_base64", "")
        
        if not b64_str:
            print(f"  [{idx}] 이미지 데이터 없음, 건너뜀")
            continue

        # (3.1) 이미지 전처리
        image_tensor = preprocess_image_bytes(b64_str)
        if image_tensor is None:
            continue

        # (3.2) 추론
        with torch.no_grad():
            output = model(image_tensor)
            pred = torch.argmax(output).item()

        # (3.3) 추론 결과 생성
        result_data = {
            "prediction": pred,
            "timestamp": datetime.now().isoformat()
        }

        # (3.4) 추론 결과를 Mobius의 inferencing_result 컨테이너에 업로드
        upload_data(MOBIUS_STUDENT_INFERENCING_RESULT_CNT_NAME, result_data, MOBIUS_STUDENT_AE_URL)

        print(f"[INFERENCE] Image {idx}: Prediction = {pred}")
        
        # 서버 처리 순서 보장을 위한 딜레이
        time.sleep(1.0)

    print("\n[완료] 모든 이미지 추론 및 업로드 완료!")
    